# Code for Sam labeling tool, takes in combined csv file (output from csv combiner tool)

In [23]:
import pandas as pd
import numpy as np
from IPython.display import FileLink
import os

import os
import glob
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull

# 1. Find all CSV files in the folder
csv_files = glob.glob('csv_for_tools/*.csv')

# 2. Error if none found
if not csv_files:
    raise FileNotFoundError("No CSV files found in 'csv_for_tools/'")

# 3. Pick the first CSV
first_csv = csv_files[0]
print(f"Reading: {first_csv}")

# 4. Read into a DataFrame
df = pd.read_csv(first_csv)

# (Optional) Inspect the top of the DataFrame
df


Reading: csv_for_tools/merged_for_tools.csv


,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,ID,UNNAMED: 8,DOUBLETS,CLUSTER POSITION,IL10,CD40,PDL1
0,61991.836,41919.664,57,µm,Surface,Position,1,3665,0,1,0,0,0,0
1,63334.027,39616.875,57,µm,Surface,Position,1,3666,0,1,0,0,0,0
2,59405.973,41580.031,57,µm,Surface,Position,1,3667,0,1,0,0,0,0
3,58870.266,40736.652,57,µm,Surface,Position,1,3668,0,1,0,0,0,0
4,58983.195,39541.340,57,µm,Surface,Position,1,3669,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130972,59469.742,39550.754,61,µm,Surface,Position,1,136598,0,0,0,0,0,0
130973,58862.957,40160.613,55,µm,Surface,Position,1,136599,0,0,0,0,0,0
130974,59356.348,39665.781,60,µm,Surface,Position,1,136600,0,0,0,0,0,0
130975,59533.316,39485.938,54,µm,Surface,Position,1,136601,0,0,0,0,0,0


In [24]:
df = df.rename(columns={'CD68- LY6G+': 'LY6G', 'CD206 OFF DP': 'CD206','F480_y':'F480'})


# Create the new "H3CIT" column, setting it to 1 if either source column is 1, else 0
df['H3CIT'] = (
    (df['H3CIT CD68- LY6G+'] == 1) |
    (df['H3CIT LY6G CLOSE TO CD68'] == 1)
).astype(int)

df.columns


KeyError: 'H3CIT CD68- LY6G+'

In [ ]:
df.columns

Index(['POSITION X', 'POSITION Y', 'POSITION Z', 'UNIT', 'CATEGORY',
       'COLLECTION', 'TIME', 'ID', 'UNNAMED: 8', 'CD3', 'CD11B', 'LY6G',
       'DOUBLETS', 'CLUSTER POSITION', 'LY6G CLOSE TO CD68',
       'H3CIT CD68- LY6G+', 'CD68', 'H3CIT LY6G CLOSE TO CD68', 'H3CIT'],
      dtype='object')

In [ ]:
# def classify_cell_old(row):
#     if row['CD11B'] == 0:
#         if row['CD3'] == 1:
#             return "t-cell"s
#         else:
#             return "DC or other cell"
#     else:  # CD11B == 1
#         if row.get('CD68', 0) == 1:
#             return "macrophage"
#         else:
#             if row['LY6G'] == 0:
#                 return "other myeloid cell"
#             elif row['LY6G'] == 1:
#                 if row['LY6C'] == 1:
#                     if row['H3CIT'] == 1:
#                         return "MDSC netting"
#                     else:
#                         return "MDSC"
#                 elif row['LY6C'] == 0:
#                     if row['H3CIT'] == 1:
#                         return "neutrophil netting"
#                     else:
#                         return "neutrophil (not netting)"


In [ ]:
def classify_cell(row):
    # Depth-first: CD11B → CD68 → LY6G → H3CIT, else CD3
    if row.get('CD11B', 0) == 1:
        # CD68 branch
        if row.get('CD68', 0) == 1:
            # LY6G in contact with macs
            if row.get('LY6G', 0) == 1:
                if row.get('H3CIT', 0) == 1:
                    return "netting neutrophil in contact with macrophage"
                else:
                    return "not netting neutrophil in contact with macrophage"
            else:
                return "macrophage"
        else:
            # CD68 == 0 branch
            if row.get('LY6G', 0) == 1:
                if row.get('H3CIT', 0) == 1:
                    return "netting neutrophil"
                else:
                    return "not netting neutrophil"
            else:
                return "other myeloid cell"
    else:
        # CD11B == 0 branch
        if row.get('CD3', 0) == 1:
            return "t cell"
        else:
            return "tumor cell / other immune cell"


In [ ]:
df['cell_type'] = df.apply(classify_cell, axis=1)


In [ ]:
df['cell_type'].value_counts()

cell_type
tumor cell / other immune cell                       61900
macrophage                                           21706
not netting neutrophil                                3331
other myeloid cell                                    2719
t cell                                                 682
netting neutrophil                                      83
not netting neutrophil in contact with macrophage        3
Name: count, dtype: int64

In [ ]:
# assuming your DataFrame is named df and columns are uppercase
# subset = df[
#     (df['CD11B'] == 1) &
#     (df['CD68']  == 1) &
#     (df['LY6G']  == 1)
# ].copy()

# subset


In [ ]:
output_dir = "csv_for_tools"
os.makedirs(output_dir, exist_ok=True)  

file_name = "merged_for_tools.csv" 
output_path = os.path.join(output_dir, file_name)

df.to_csv(output_path, index=False)

FileLink(output_path)

/Users/patrick/Desktop2/lab_proj1/csv_for_tools/merged_for_tools.csv